In [ ]:
import numpy as np
import gdspy
import matplotlib.pyplot as plt

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2
import snakenmake
from snakenmake import *

In [ ]:
def plot_cell(cell, exclude=(2,)):
    # FROM: https://github.com/heitzmann/gdspy/issues/42
    poly_dict = cell.get_polygons(by_spec=True)
    plt.figure(figsize=(20, 12))
    for layer_datatype, polys in poly_dict.items():
        if layer_datatype[0] in exclude:
            continue
        for poly in polys:
            plt.fill(*poly.T, lw=0.5, ec="k", fc=(1, 0, 0, 0.5))
    plt.axes().set_aspect("equal", "datalim")


def write_gds(main_cell, filename, unit=1.0e-6, precision=1.0e-9):
    writer = gdspy.GdsWriter(filename, unit=unit, precision=precision)
    for cell in [main_cell] + list(main_cell.get_dependencies(True)):
        writer.write_cell(cell)
    writer.close()

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
chip_dims = np.array([23e3, 13e3])

In [ ]:
fov = 6.5 * 2048 / 40
fov

In [ ]:
23e3 * 13e3 / fov**2

# Flow calculations

In [ ]:
import pint

u = pint._DEFAULT_REGISTRY

In [ ]:
eta = 1 * u.centipoise

In [ ]:
def Q_over_deltap(eta, L, h, w):
    h_ = np.minimum(h, w)
    w_ = np.maximum(h, w)
    h, w = h_, w_
    return h**3 * w / (12 * eta * L) * (1 - 0.630 * h / w)

In [ ]:
baseline_md = chip_metadata(feeding_channel_width=150, lane_gap=400)
print(baseline_md)

In [ ]:
h = 50 * u.um
w = baseline_md["feeding_channel_width"] * u.um
L = baseline_md["split"][0] * baseline_md["lane_length"] * u.um
baseline_Q_over_deltap = Q_over_deltap(eta, L, h, w)

In [ ]:
h = 50 * u.um
w = np.linspace(50, 150, 100) * u.um
L = 15 * 17000 * u.um
plt.plot(w, baseline_Q_over_deltap / Q_over_deltap(eta, L, h, w))

In [ ]:
plt.figure(figsize=(10, 8))
hs = np.array([50, 75, 100, 150]) * u.um
w = np.linspace(50, 150, 100) * u.um
chips = [
    chip_metadata(feeding_channel_width=width.magnitude, trench_spacing=2, split=3)
    for width in w
]
L = np.array([np.max(m["split"]) * m["lane_length"] for m in chips]) * u.um
# L = 15 * 17000 * u.um
eta = 1 * u.centipoise
ax = plt.gca()
for h in hs:
    ax.plot(
        w,
        baseline_Q_over_deltap / Q_over_deltap(eta, L, h, w),
        label="feeding channel height: {:~P}".format(h),
    )
ax2 = plt.gca().twinx()
ax2.plot(
    w,
    [m["num_trenches"] for m in chips],
    ls="-.",
    c="purple",
    label="number of trenches (2 um trench spacing)",
)
# ax2.plot(w, [np.max(m['split']) for m in chips], c='r', label='number of lanes')
ax.axhline(1, c="k", ls=":", label="15 lane snake (50 um height)")
ax.legend()
ax.set_ylabel("pressure (relative to 15-lane snake)")
ax.set_xlabel("feeding channel width (um)")
ax2.legend(loc="lower left")

# Designs

In [ ]:
def chip_metadata(**kwargs):
    metadata = {}
    main_cell = chip(
        "my_chip", **{"metadata": metadata, **kwargs, "draw_trenches": False}
    )
    return metadata["my_chip"]

In [ ]:
from snakenmake import *

metadata = {}
main_cell = chip(
    "Big Snake",
    snake,
    split=(7, 13, 17, 27, 5),
    feeding_channel_width=65,
    trench_spacing=1,
    lane_gap=17,
    draw_trenches=False,
    metadata=metadata,
)
print(metadata)
plot_cell(main_cell)

In [ ]:
%%time
from snakenmake import *

metadata = {}
main_cell = wafer(
    [
        chip(
            "Big Snake S3 FC{fc} L35 W1.5 TS{ts} LG{lg}".format(fc=fc, ts=ts, lg=lg),
            snake,
            split=3,
            feeding_channel_width=fc,
            trench_spacing=ts,
            lane_gap=lg,
            tick_labels=False,
            text=False,
            draw_trenches=True,
            metadata=metadata,
        )
        for (fc, lg) in zip((90, 45), (20, 17))
        for ts in (1, 1.5, 2)
    ],
    "Big Snakes \n BS/JQS 180513",
    text=False,
)
write_gds(main_cell, "bigsnakes4_notext.gds")

In [ ]:
%%time
from snakenmake import *

metadata = {}
main_cell = wafer(
    [
        chip(
            "Big Snake S3 FC{fc} L35 W1.5 TS{ts} LG{lg}".format(fc=fc, ts=ts, lg=lg),
            snake,
            split=3,
            feeding_channel_width=fc,
            trench_spacing=ts,
            lane_gap=lg,
            tick_labels=False,
            text=True,
            draw_trenches=True,
            metadata=metadata,
        )
        for (fc, lg) in zip((90, 45), (20, 17))
        for ts in (1, 1.5, 2)
    ],
    "Big Snakes \n BS/JQS 180513",
    text=True,
)
write_gds(main_cell, "bigsnakes4_noticklabels.gds")

In [ ]:
!du -hs bigsnakes4_noticklabels.gds

In [ ]:
!du -hs bigsnakes4_notext.gds

In [ ]:
metadata

In [ ]:
%%time
from snakenmake import *

metadata = {}
main_cell = wafer(
    [
        chip(
            "Big Snake S3 FC{fc} L35 W1.5 TS{ts} LG20".format(fc=fc, ts=ts),
            snake,
            split=3,
            feeding_channel_width=fc,
            trench_spacing=ts,
            draw_trenches=True,
            tick_labels=False,
            metadata=metadata,
        )
        for fc in (90,)
        for ts in (1, 1.5, 2)
    ],
    "Big Snakes Backup \n BS/JQS 180513",
)
write_gds(main_cell, "bigsnakes3backup.gds")